# Předpřipravený tagger

In [ ]:
cs = "Proč chceš stát na dešti? Není o co stát."
en = "They refuse to permit us to obtain the refuse permit."

In [ ]:
import nltk

In [ ]:
en_tok = nltk.word_tokenize(en)
en_tok

In [ ]:
nltk.pos_tag(en_tok)

In [ ]:
from corpy.morphodita import Tagger

In [ ]:
czech_pos_tagger = Tagger("/home/lukes/edu/python/czech-morfflex-pdt-161115/czech-morfflex-pdt-161115-pos_only.tagger")

In [ ]:
list(czech_pos_tagger.tag(cs, sents=True))

# Otagovaný korpus

In [ ]:
nltk.corpus.brown.tagged_words()

In [ ]:
nltk.corpus.brown.tagged_words(tagset="universal")

Možnost načíst si věty z otagovaného korpusu budeme potřebovat při trénování a testování našich vlastních taggerů (testování = porovnávání výstupu našeho taggeru s ručně označkovaným, tzv. **zlatým** standardem).

## Trénovací a testovací data

Tagger je potřeba trénovat a testovat na různých sadách dat, jinak by se nám mohlo stát, že vytvoříme tagger, který se do detailu naučí všechna specifika našich trénovacích dat a bude na nich velmi úspěšný, ale nezjistíme, že špatně generalizuje a že na jakémkoli jiném vstupu je jeho úspěšnost mnohem horší.

In [ ]:
!head ~/edu/python/pdt3.*.vrt

In [ ]:
def vert_sents(path):
    """Načte korpus jako seznam seznamů (= vět) ntic (= pozic)."""
    sents = []
    with open(path) as file:
        for line in file:
            line = line.strip("\n")
            if line == "<s>":
                sent = []
            elif line == "</s>":
                sents.append(sent)
            elif "\t" in line:
                word, _, tag = line.split("\t")
                pos = tag[0]
                sent.append((word, pos))
    return sents    

In [ ]:
train = vert_sents("/home/lukes/edu/python/pdt3.train.vrt")
test = vert_sents("/home/lukes/edu/python/pdt3.test.vrt")

In [ ]:
train[0:2]

In [ ]:
test[0:2]

# Vlastní tagger

Jak k problému vůbec přistoupit? Na jakém principu by měl tagger fungovat? Představte si, že chcete vymyslet formální postup, podle něhož by bez znalosti daného jazyka mohl značkování provést i člověk, ale místo člověku ho pak povíte počítači. Nesnažte se rovnou vyřešit všechno, zvolte "inženýrský" přístup -- přednostně se snažte identifikovat a řešit části problému, které mají při minimálním vynaloženém úsilí maximální účinek. Účinek v tomto případě měříme úspěšností taggeru -- procentem značek přidělených tak, aby odpovídaly referenčnímu korpusu (zlatému standardu).

# Nejčastější POS

In [ ]:
pos_fdist = nltk.FreqDist(p for s in train for _, p in s)

In [ ]:
def default_tagger(word):
    return pos_fdist.max()

In [ ]:
default_tagger("kočka")

In [ ]:
default_tagger("sadlfkas")

# Nejčastější POS pro daný tvar

In [ ]:
unigram_cfdist = nltk.ConditionalFreqDist((w, p) for s in train for w, p in s)

In [ ]:
unigram_cfdist["včera"].max()

In [ ]:
unigram_cfdist["stát"].max()

In [ ]:
unigram_cfdist["asdfsaf"].max()

# Přes regulární výrazy

In [ ]:
regex_tagger = nltk.RegexpTagger([
    (r".*ání$", "N"),
    (r".*[áí]t$", "V"),
    (r"a|i|(proto|tak)?že|když|aby|nebo|ani", "J"),
], backoff=nltk.DefaultTagger("N"))

In [ ]:
regex_tagger.tag(["tání"])

In [ ]:
regex_tagger.tag(["kočka"])

In [ ]:
regex_tagger.evaluate(list(vert_sents("/home/lukes/edu/python/pdt3.test.vrt")))

# N-gramový tagger

N-gramové značkování funguje tak, že počítač naučíme rozpoznávat pravidelnosti v tom, jaké morfologické značky se nejčastěji objevují ve kterých kontextech. Kontextem zde míníme typicky aktuální tvar, který se snažíme označkovat, a sekvenci předchozích již stanovených značek. Např. pro trigramový tagger, který za kontext považuje aktuální tvar + značky na předchozích **dvou** pozicích (spolu s aktuální to dává tři, proto **tri**gramový) si můžeme situaci znázornit následovně:

![n-gram tagger](http://www.nltk.org/images/tag-context.png)

Pokud se taková data snažíme reprezentovat v Pythonu, dobrou volbou může být podmíněná frekvenční distribuce (`nltk.ConditionalFreqDist`). Podmínky odpovídají jednotlivým kontextům a uvnitř každé podmínky máme frekvenční distribuci morfologických značek, které jsme v daném kontextu zaznamenali. Z takovýchto dat jsme pak schopni určit, že např. pro kontext `(aktuální_slovo="pláče", předcházející_tag="R (předložka)")` je nepravděpodobnější tag (pro aktuální slovo, tj. "pláče") `N (substantivum)`.

Některé kontexty budou jednoznačné, tj. frekvenční distribuce v tomto kontextu přípustných tagů bude obsahovat jen jeden prvek. Ale určitá část (i v závislosti na velikosti trénovacích dat) bude patrně víceznačná:

In [ ]:
bigram_cfdist = nltk.ConditionalFreqDist(((b1.pos, b2.word), b2.pos) for s in vert_sents("/home/lukes/edu/python/pdt3.train.vrt") for b1, b2 in nltk.bigrams(s))

In [ ]:
for k, v in bigram_cfdist.items():
    if len(v) > 1:
        print(k, dict(v))

## N-gramové taggery v NLTK

N-gramové taggery lze v NLTK natrénovat velmi jednoduše a pospojovat je dohromady pomocí tzv. "backoffu". To je metoda, která řeší nedostatek dat ("data sparsity"): pokud nemůžeme pro aktuální kontext vybrat značku na základě bigramového taggeru, protože jsme tento kontext v trénovacích datech jednoduše nepotkali, zkusíme unigramový; pokud selže i ten (= slovní tvar jsme v trénovacích datech prostě ani jednou nepotkali), zvolíme pro pozici nějakou defaultní značku (typicky nejčastější slovní druh).

In [ ]:
t2 = nltk.BigramTagger(train)
# metoda evaluate spočítá úspěšnost taggeru -- zde je potřeba použít jiná
# data než trénovací, aby byl údaj spolehlivý
t2.evaluate(test)

In [ ]:
t0 = nltk.DefaultTagger("N")
t1 = nltk.UnigramTagger(train, backoff=t0)
t2 = nltk.BigramTagger(train, backoff=t1)
t2.evaluate(test)

## Vlastní tagger

Abychom se trochu pocvičili v Pythonu, zkusili jsme si napsat vlastní tagger(y). Celá metoda je (jako většina metod v NLP) založená na chytrém využití frekvencí: spočítáme frekvence kontextů a odpovídajících značek v trénovacích datech a pak se z nich snažíme zpětně odvodit odpovídající značky pro kontexty, které potkáváme při značkování nových vět.

In [ ]:
# frekvenční distribuce: počet výskytů různých jevů, např. slov v textu
nltk.FreqDist(["kočka", "stát", "kočka"])

In [ ]:
# podmíněná frekvenční distribuce: počet výskytů různých jevů za různých podmínek,
# např. slovnědruhových značek v kontextu slovního tvaru, který se aktuálně snažíme
# opatřit značkou
cfd = nltk.ConditionalFreqDist([("stát", "N"), ("kočka", "N"), ("stát", "V"), ("stát", "V")])
cfd

In [ ]:
# pokud se díváme na tvar "stát", tak na základě "trénovacích dat" (= těch čtyř
# pozic v předchozí buňce) je nejpravděpodobnější značka "V"
cfd["stát"].max()

In [ ]:
# u tvaru, který jsme neviděli, nelze stanovit `.max()`, dojde tedy k chybě
cfd["bambule"].max()

In [ ]:
# chybu lze odchytit pomocí syntaktického konstruktu `try ... except`
word = "bambule"
try:
    cfd[word].max()
except ValueError:
    print(f"Na slovo {word} jsme bohužel v trénovacích datech nenarazili :(")

### Unigramový tagger

In [ ]:
def train_uni(data):
    """Natrénuje unigramový tagger."""
    cfd = nltk.ConditionalFreqDist()
    for sent in data:
        for word, tag in sent:
            cfd[word.lower()][tag] += 1
    return cfd

# též možno zapsat úsporněji takto
def train_uni2(data):
    """Natrénuje unigramový tagger."""
    return nltk.ConditionalFreqDist(
        (word.lower(), tag)
        for sent in data
        for (word, tag) in sent
    )

In [ ]:
def tag_uni(sent, uni_cfd):
    """Otaguje větu pomocí unigramového taggeru."""
    tagged_sent = []
    for word in sent:
        try:
            tag = uni_cfd[word.lower()].max()
        except ValueError:
            # pokud tvar neznáme, přiřadíme mu tag "N"
            tag = "N"
        tagged_sent.append((word, tag))
    return tagged_sent

In [ ]:
uni_tagger = train_uni(train)

In [ ]:
# s jakými tagy jsme v trénovacích datech viděli tvar "V"?
uni_tagger["V"]

In [ ]:
# s jakými tagy jsme v trénovacích datech viděli tvar "v"?
uni_tagger["v"]

In [ ]:
tag_uni("Už zase frunakulózně prší .".split(), uni_tagger)

### Bigramový tagger

In [ ]:
# kontext může být i složitější, třeba předchozí tag + aktuální tvar;
# v takovém případě ho budeme reprezentovat jako ntici
context = ("A", "stát")
tag = "N"
cfd = nltk.ConditionalFreqDist([(context, tag)])

In [ ]:
cfd[("A", "stát")]

In [ ]:
cfd[("A", "stát")].max()

In [ ]:
cfd[("R", "stát")]

In [ ]:
# taky budeme potřebovat mít možnost nějak pohodlně procházet věty po
# bigramech a zachytit, která slova jsme potkali na začátcích vět;
# s tím prvním nám pomůže funkce `nltk.bigrams()` (pozor, vrací
# generátor!), s tím druhým si poradíme tak, že na začátek věty vždy
# přilepíme falešnou pozici s nějakými speciálními hodnotami pro tvar
# a značku, podle nichž se pozná začátek věty
sent = train[0]
sent

In [ ]:
# napadne vás důvod, proč zde vytváříme konkatenací nový seznam, místo
# abychom do toho stávajícího vložili na začátek ntici `(None, None)`
# (tj. `sent.insert(0, (None, None))`)?
sent = [(None, None)] + sent

In [ ]:
for i, (b1, b2) in enumerate(nltk.bigrams(sent)):
    print(f"Bigram č. {i+1}\nPrvní půlka bigramu: {b1}\nDruhá půlka bigramu: {b2}\n")

In [ ]:
def train_bi(data):
    """Natrénuje bigramový tagger."""
    cfd = nltk.ConditionalFreqDist()
    for sent in data:
        sent = [(None, None)] + sent
        for (_, previous_tag), (current_word, current_tag) in nltk.bigrams(sent):
            cfd[(previous_tag, current_word.lower())][current_tag] += 1
    return cfd

In [ ]:
bi_tagger = train_bi(train)

In [ ]:
# tagy, které jsme viděli pro slovní tvar "v" na začátku věty (= předchozí
# tag je `None`)
bi_tagger[(None, "v")]

In [ ]:
for (prev_tag, curr_word), fd in bi_tagger.items():
    if prev_tag is not None and curr_word == "v":
        print(f"V kontextu předcházející značky {prev_tag} má tvar 'v' následující "
              f"frekvenční distribuci možných značek:\n{fd!r}\n")

Jak by mohla tedy vypadat funkce `tag_bi()`, která otaguje větu pomocí bigramového modelu? (Řešení viz poznámky z hodiny.)